# FAISS 高级搜索演示
# FAISS Advanced Search Demo

这个笔记本演示如何使用FAISS库进行高效的大规模向量搜索。

In [ ]:
# 导入必要的库
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import time
from src.advanced_search import FAISSSearch, AnnoySearch
from src.basic_vector_search import BasicVectorSearch

## 1. 生成测试数据

In [ ]:
# 生成大规模测试数据
np.random.seed(42)

# 不同规模的数据集
small_data = np.random.randn(1000, 128)    # 1K向量
medium_data = np.random.randn(10000, 128)  # 10K向量
large_data = np.random.randn(50000, 128)   # 50K向量

print(f"小规模数据: {small_data.shape}")
print(f"中等规模数据: {medium_data.shape}")
print(f"大规模数据: {large_data.shape}")

# 查询向量
query_vectors = np.random.randn(10, 128)
print(f"查询向量: {query_vectors.shape}")

## 2. FAISS vs 基础搜索性能比较

In [ ]:
def benchmark_search_methods(data, query_vectors, k=10):
    """比较不同搜索方法的性能"""
    results = {}
    
    # 基础搜索
    print("测试基础搜索...")
    basic_search = BasicVectorSearch()
    basic_search.add_vectors(data)
    
    start_time = time.time()
    for query in query_vectors:
        _ = basic_search.search(query, k=k)
    basic_time = time.time() - start_time
    results['基础搜索'] = basic_time
    
    # FAISS搜索 (如果可用)
    try:
        print("测试FAISS搜索...")
        faiss_search = FAISSSearch(dimension=data.shape[1])
        faiss_search.add_vectors(data)
        
        start_time = time.time()
        _ = faiss_search.batch_search(query_vectors, k=k)
        faiss_time = time.time() - start_time
        results['FAISS'] = faiss_time
    except ImportError:
        print("FAISS不可用，跳过测试")
        results['FAISS'] = None
    
    # Annoy搜索 (如果可用)
    try:
        print("测试Annoy搜索...")
        annoy_search = AnnoySearch(dimension=data.shape[1])
        annoy_search.add_vectors(data)
        annoy_search.build_index()
        
        start_time = time.time()
        for query in query_vectors:
            _ = annoy_search.search(query, k=k)
        annoy_time = time.time() - start_time
        results['Annoy'] = annoy_time
    except ImportError:
        print("Annoy不可用，跳过测试")
        results['Annoy'] = None
    
    return results

# 在不同数据规模上进行测试
datasets = {
    '小规模(1K)': small_data,
    '中等规模(10K)': medium_data,
    '大规模(50K)': large_data
}

benchmark_results = {}
for name, data in datasets.items():
    print(f"\n=== {name} 数据集测试 ===")
    benchmark_results[name] = benchmark_search_methods(data, query_vectors[:5])  # 使用5个查询

## 3. 结果可视化

In [ ]:
# 可视化性能比较结果
methods = ['基础搜索', 'FAISS', 'Annoy']
dataset_names = list(benchmark_results.keys())

# 准备数据
performance_data = {method: [] for method in methods}

for dataset_name in dataset_names:
    for method in methods:
        time_taken = benchmark_results[dataset_name].get(method)
        performance_data[method].append(time_taken if time_taken is not None else 0)

# 绘制性能比较图
plt.figure(figsize=(12, 6))

x = np.arange(len(dataset_names))
width = 0.25

for i, method in enumerate(methods):
    times = performance_data[method]
    if any(t > 0 for t in times):  # 只绘制有数据的方法
        plt.bar(x + i * width, times, width, label=method)

plt.xlabel('数据集规模')
plt.ylabel('搜索时间 (秒)')
plt.title('不同搜索方法性能比较')
plt.xticks(x + width, dataset_names)
plt.legend()
plt.yscale('log')  # 使用对数坐标
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 打印具体数值
print("\n性能比较结果 (秒):")
print("-" * 50)
for dataset_name in dataset_names:
    print(f"\n{dataset_name}:")
    for method in methods:
        time_taken = benchmark_results[dataset_name].get(method)
        if time_taken is not None:
            print(f"  {method}: {time_taken:.4f}秒")
        else:
            print(f"  {method}: 不可用")

## 4. FAISS 不同索引类型比较

In [ ]:
# 如果FAISS可用，测试不同索引类型
try:
    import faiss
    
    print("测试FAISS不同索引类型...")
    
    # 使用中等规模数据
    test_data = medium_data
    test_queries = query_vectors[:3]
    
    index_types = {
        'Flat (精确)': 'Flat',
        'IVF100 (近似)': 'IVF100,Flat', 
        'LSH (哈希)': 'LSH'
    }
    
    faiss_results = {}
    
    for name, index_type in index_types.items():
        try:
            print(f"\n测试 {name}...")
            
            # 创建索引
            dimension = test_data.shape[1]
            if index_type == 'Flat':
                index = faiss.IndexFlatL2(dimension)
            elif index_type == 'IVF100,Flat':
                quantizer = faiss.IndexFlatL2(dimension)
                index = faiss.IndexIVFFlat(quantizer, dimension, 100)
                index.train(test_data.astype('float32'))
            elif index_type == 'LSH':
                index = faiss.IndexLSH(dimension, 256)
            
            # 添加向量
            index.add(test_data.astype('float32'))
            
            # 搜索测试
            start_time = time.time()
            distances, indices = index.search(test_queries.astype('float32'), 10)
            search_time = time.time() - start_time
            
            faiss_results[name] = {
                'time': search_time,
                'memory': index.ntotal * 4 * dimension / (1024**2)  # 估算内存使用(MB)
            }
            
            print(f"  搜索时间: {search_time:.4f}秒")
            print(f"  估算内存: {faiss_results[name]['memory']:.1f}MB")
            
        except Exception as e:
            print(f"  {name} 测试失败: {e}")
    
    # 可视化FAISS索引比较
    if faiss_results:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        names = list(faiss_results.keys())
        times = [faiss_results[name]['time'] for name in names]
        memories = [faiss_results[name]['memory'] for name in names]
        
        # 搜索时间比较
        ax1.bar(names, times)
        ax1.set_ylabel('搜索时间 (秒)')
        ax1.set_title('FAISS索引类型 - 搜索时间')
        ax1.tick_params(axis='x', rotation=45)
        
        # 内存使用比较
        ax2.bar(names, memories)
        ax2.set_ylabel('内存使用 (MB)')
        ax2.set_title('FAISS索引类型 - 内存使用')
        ax2.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()

except ImportError:
    print("FAISS不可用，跳过详细测试")
    print("安装FAISS: pip install faiss-cpu")

## 5. 实际应用场景模拟

In [ ]:
# 模拟实际应用场景：文档搜索系统
print("=== 文档搜索系统模拟 ===")

# 生成模拟文档向量（表示文档特征）
num_documents = 10000
vector_dim = 384  # 典型的sentence embedding维度

# 创建一些聚类中心（代表不同主题）
num_clusters = 20
cluster_centers = np.random.randn(num_clusters, vector_dim)

# 生成围绕聚类中心的文档向量
document_vectors = []
document_labels = []

for i in range(num_documents):
    cluster_id = np.random.randint(0, num_clusters)
    # 在聚类中心附近生成向量
    noise = np.random.randn(vector_dim) * 0.3
    doc_vector = cluster_centers[cluster_id] + noise
    document_vectors.append(doc_vector)
    document_labels.append(f"主题{cluster_id}")

document_vectors = np.array(document_vectors)

print(f"生成了 {num_documents} 个文档向量，分属 {num_clusters} 个主题")
print(f"向量维度: {vector_dim}")

# 创建搜索引擎
search_engines = {}

# 基础搜索
basic_engine = BasicVectorSearch()
basic_engine.add_vectors(document_vectors)
search_engines['基础搜索'] = basic_engine

# FAISS搜索 (如果可用)
try:
    faiss_engine = FAISSSearch(dimension=vector_dim)
    faiss_engine.add_vectors(document_vectors)
    search_engines['FAISS'] = faiss_engine
except:
    print("FAISS不可用")

# Annoy搜索 (如果可用)
try:
    annoy_engine = AnnoySearch(dimension=vector_dim)
    annoy_engine.add_vectors(document_vectors)
    annoy_engine.build_index()
    search_engines['Annoy'] = annoy_engine
except:
    print("Annoy不可用")

print(f"\n可用搜索引擎: {list(search_engines.keys())}")

In [ ]:
# 执行搜索测试
query_vector = np.random.randn(vector_dim)
k = 10

print("\n=== 搜索结果比较 ===")

for engine_name, engine in search_engines.items():
    print(f"\n{engine_name} 搜索结果:")
    
    start_time = time.time()
    
    if hasattr(engine, 'batch_search'):
        # FAISS批量搜索
        results = engine.batch_search([query_vector], k=k)[0]
    else:
        # 单个搜索
        results = engine.search(query_vector, k=k)
    
    search_time = time.time() - start_time
    
    print(f"  搜索时间: {search_time:.4f}秒")
    print(f"  前5个结果:")
    
    for i, (idx, score) in enumerate(results[:5]):
        label = document_labels[idx]
        print(f"    {i+1}. 文档{idx} ({label}), 得分: {score:.3f}")

## 6. 总结和建议

### 性能特点:
- **基础搜索**: 精确但慢，适合小规模数据
- **FAISS**: 快速且灵活，适合大规模数据
- **Annoy**: 内存效率高，适合只读场景

### 选择建议:
- **< 10K向量**: 基础搜索足够
- **10K-1M向量**: 使用FAISS
- **> 1M向量**: 使用FAISS IVF索引
- **内存限制**: 考虑Annoy

下一个笔记本将展示完整的应用系统。

# FAISS高性能向量搜索演示

FAISS (Facebook AI Similarity Search) 是Facebook开发的高效相似性搜索库，专门用于大规模向量搜索。

## 学习目标
- 理解FAISS的工作原理和优势
- 学习不同的FAISS索引类型
- 实践FAISS在大规模向量搜索中的应用
- 性能对比和优化技巧

In [ ]:
# 导入必要的库
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import faiss
from sklearn.datasets import make_blobs
from sklearn.preprocessing import normalize

from src.advanced_search import FAISSSearch, AnnoySearch
from src.text_vectorizer import TextVectorizer
from src.utils import load_documents, VectorSearchBenchmark

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False   # 用来正常显示负号

## 1. FAISS基础概念

FAISS提供多种索引类型，每种都有不同的特点和适用场景。

In [ ]:
# 创建示例数据集
def create_sample_vectors(n_samples=1000, n_features=128, n_clusters=5):
    """创建示例向量数据集"""
    X, y = make_blobs(n_samples=n_samples, centers=n_clusters, 
                      n_features=n_features, random_state=42)
    # 归一化向量（对于余弦相似度很重要）
    X = normalize(X)
    return X.astype('float32'), y

# 生成测试数据
vectors, labels = create_sample_vectors(n_samples=10000, n_features=128)
print(f"生成了 {vectors.shape[0]} 个 {vectors.shape[1]} 维向量")
print(f"向量类型: {vectors.dtype}")
print(f"向量范围: [{vectors.min():.3f}, {vectors.max():.3f}]")

## 2. FAISS索引类型介绍

让我们演示不同类型的FAISS索引及其特点。

In [ ]:
# 演示不同的FAISS索引类型
dimension = vectors.shape[1]
n_vectors = vectors.shape[0]

print("FAISS索引类型演示:")
print("="*50)

# 1. Flat索引 - 精确搜索
print("\n1. Flat索引 (精确搜索):")
index_flat = faiss.IndexFlatL2(dimension)
index_flat.add(vectors)
print(f"   索引中的向量数: {index_flat.ntotal}")
print(f"   是否训练: {index_flat.is_trained}")

# 2. IVF索引 - 倒排文件索引
print("\n2. IVF索引 (近似搜索):")
nlist = 100  # 聚类中心数
quantizer = faiss.IndexFlatL2(dimension)
index_ivf = faiss.IndexIVFFlat(quantizer, dimension, nlist)
print(f"   训练前状态: {index_ivf.is_trained}")
index_ivf.train(vectors)
print(f"   训练后状态: {index_ivf.is_trained}")
index_ivf.add(vectors)
print(f"   索引中的向量数: {index_ivf.ntotal}")
print(f"   聚类中心数: {nlist}")

# 3. HNSW索引 - 分层可导航小世界图
print("\n3. HNSW索引 (图-based搜索):")
index_hnsw = faiss.IndexHNSWFlat(dimension, 32)  # 32是M参数
index_hnsw.add(vectors)
print(f"   索引中的向量数: {index_hnsw.ntotal}")
print(f"   HNSW M参数: 32")

## 3. 搜索性能比较

比较不同索引类型的搜索速度和准确性。

In [ ]:
# 性能测试函数
def benchmark_index(index, query_vectors, k=10, name="Unknown"):
    """测试索引性能"""
    start_time = time.time()
    distances, indices = index.search(query_vectors, k)
    search_time = time.time() - start_time
    
    return {
        'name': name,
        'search_time': search_time,
        'qps': len(query_vectors) / search_time,  # 每秒查询数
        'avg_distance': np.mean(distances),
        'distances': distances,
        'indices': indices
    }

# 准备查询向量
n_queries = 100
query_vectors = vectors[:n_queries].copy()
k = 10  # 返回前10个最相似的

print(f"性能测试: {n_queries} 个查询，每个返回 {k} 个结果")
print("="*60)

# 测试所有索引
indices_to_test = [
    (index_flat, "Flat (精确)"),
    (index_ivf, "IVF (近似)"),
    (index_hnsw, "HNSW (图-based)")
]

results = []
for index, name in indices_to_test:
    result = benchmark_index(index, query_vectors, k, name)
    results.append(result)
    print(f"\n{name}:")
    print(f"  搜索时间: {result['search_time']:.4f} 秒")
    print(f"  QPS: {result['qps']:.1f} 查询/秒")
    print(f"  平均距离: {result['avg_distance']:.4f}")

In [ ]:
# 可视化性能比较
plt.figure(figsize=(15, 5))

# 搜索时间比较
plt.subplot(1, 3, 1)
names = [r['name'] for r in results]
times = [r['search_time'] for r in results]
bars1 = plt.bar(names, times, color=['blue', 'orange', 'green'], alpha=0.7)
plt.title('搜索时间比较')
plt.ylabel('时间 (秒)')
plt.xticks(rotation=45)
for bar, time in zip(bars1, times):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{time:.3f}s', ha='center', va='bottom')

# QPS比较
plt.subplot(1, 3, 2)
qps_values = [r['qps'] for r in results]
bars2 = plt.bar(names, qps_values, color=['blue', 'orange', 'green'], alpha=0.7)
plt.title('查询吞吐量比较')
plt.ylabel('QPS (查询/秒)')
plt.xticks(rotation=45)
for bar, qps in zip(bars2, qps_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50,
             f'{qps:.0f}', ha='center', va='bottom')

# 距离分布比较
plt.subplot(1, 3, 3)
for result in results:
    plt.hist(result['distances'].flatten(), bins=30, alpha=0.5, 
             label=result['name'], density=True)
plt.title('距离分布比较')
plt.xlabel('距离')
plt.ylabel('密度')
plt.legend()

plt.tight_layout()
plt.show()

## 4. 准确性分析

比较近似搜索方法与精确搜索的准确性。

In [ ]:
def calculate_recall(true_neighbors, found_neighbors, k):
    """计算召回率"""
    recalls = []
    for i in range(len(true_neighbors)):
        true_set = set(true_neighbors[i][:k])
        found_set = set(found_neighbors[i][:k])
        recall = len(true_set.intersection(found_set)) / k
        recalls.append(recall)
    return np.mean(recalls)

# 使用Flat索引作为ground truth
true_distances, true_indices = index_flat.search(query_vectors, k)

print("准确性分析 (以Flat索引为基准):")
print("="*40)

for result in results[1:]:  # 跳过Flat索引本身
    recall = calculate_recall(true_indices, result['indices'], k)
    print(f"\n{result['name']}:")
    print(f"  召回率@{k}: {recall:.3f}")
    print(f"  速度提升: {results[0]['search_time']/result['search_time']:.1f}x")

## 5. 实际文本搜索应用

在真实的文本数据上演示FAISS的应用。

In [ ]:
# 加载真实文档数据
documents = load_documents('../data/sample_documents.json')
vectorizer = TextVectorizer()

print("加载文档数据...")
texts = [doc['content'] for doc in documents]

# 使用Sentence Transformers生成向量（如果可用）
try:
    print("使用Sentence Transformers生成向量...")
    doc_vectors = vectorizer.sentence_transformer_vectorize(texts)
except:
    print("Sentence Transformers不可用，使用TF-IDF...")
    doc_vectors = vectorizer.tfidf_vectorize(texts)
    # 转换为float32并归一化
    doc_vectors = normalize(doc_vectors.toarray().astype('float32'))

print(f"文档向量维度: {doc_vectors.shape}")

In [ ]:
# 使用我们的FAISSSearch类
print("初始化FAISS搜索引擎...")
faiss_search = FAISSSearch(vector_dim=doc_vectors.shape[1])
faiss_search.add_vectors(doc_vectors)

print(f"FAISS索引中的向量数: {faiss_search.index.ntotal}")

# 测试搜索
query_text = "人工智能技术发展趋势"
print(f"\n搜索查询: '{query_text}'")

# 生成查询向量
try:
    query_vector = vectorizer.sentence_transformer_vectorize([query_text])
except:
    query_vector = vectorizer.tfidf_vectorize([query_text])
    query_vector = normalize(query_vector.toarray().astype('float32'))

# 执行搜索
start_time = time.time()
distances, indices = faiss_search.search(query_vector, k=5)
search_time = time.time() - start_time

print(f"\n搜索结果 (耗时: {search_time:.4f}秒):")
print("="*50)
for i, (idx, dist) in enumerate(zip(indices[0], distances[0])):
    if idx < len(documents):
        doc = documents[idx]
        print(f"{i+1}. {doc['title']}")
        print(f"   类别: {doc['category']}")
        print(f"   距离: {dist:.4f}")
        print(f"   内容: {doc['content'][:100]}...")
        print()

## 6. 大规模数据性能测试

测试FAISS在大规模数据上的性能表现。

In [ ]:
# 生成不同规模的数据集进行测试
scales = [1000, 5000, 10000, 50000]
dimension = 128
results_scale = []

print("大规模性能测试:")
print("="*40)

for scale in scales:
    print(f"\n测试规模: {scale:,} 向量")
    
    # 生成数据
    large_vectors, _ = create_sample_vectors(n_samples=scale, n_features=dimension)
    
    # 创建索引并添加向量
    start_time = time.time()
    index = faiss.IndexFlatL2(dimension)
    index.add(large_vectors)
    build_time = time.time() - start_time
    
    # 测试搜索性能
    query_vecs = large_vectors[:100]  # 使用前100个作为查询
    start_time = time.time()
    distances, indices = index.search(query_vecs, 10)
    search_time = time.time() - start_time
    
    result = {
        'scale': scale,
        'build_time': build_time,
        'search_time': search_time,
        'qps': 100 / search_time
    }
    results_scale.append(result)
    
    print(f"  索引构建时间: {build_time:.3f}秒")
    print(f"  搜索时间: {search_time:.4f}秒")
    print(f"  QPS: {result['qps']:.1f}")

In [ ]:
# 可视化规模性能关系
plt.figure(figsize=(15, 5))

scales_list = [r['scale'] for r in results_scale]
build_times = [r['build_time'] for r in results_scale]
search_times = [r['search_time'] for r in results_scale]
qps_list = [r['qps'] for r in results_scale]

# 构建时间 vs 规模
plt.subplot(1, 3, 1)
plt.plot(scales_list, build_times, 'o-', color='blue', linewidth=2)
plt.title('索引构建时间 vs 数据规模')
plt.xlabel('向量数量')
plt.ylabel('构建时间 (秒)')
plt.grid(True, alpha=0.3)

# 搜索时间 vs 规模
plt.subplot(1, 3, 2)
plt.plot(scales_list, search_times, 'o-', color='orange', linewidth=2)
plt.title('搜索时间 vs 数据规模')
plt.xlabel('向量数量')
plt.ylabel('搜索时间 (秒)')
plt.grid(True, alpha=0.3)

# QPS vs 规模
plt.subplot(1, 3, 3)
plt.plot(scales_list, qps_list, 'o-', color='green', linewidth=2)
plt.title('查询吞吐量 vs 数据规模')
plt.xlabel('向量数量')
plt.ylabel('QPS (查询/秒)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. FAISS高级功能

演示FAISS的一些高级功能，如索引持久化、量化等。

In [ ]:
# 索引持久化
print("索引持久化演示:")
print("="*30)

# 创建一个索引
test_vectors, _ = create_sample_vectors(1000, 64)
index = faiss.IndexFlatL2(64)
index.add(test_vectors)

print(f"原始索引中的向量数: {index.ntotal}")

# 保存索引
index_path = '../data/faiss_index.bin'
faiss.write_index(index, index_path)
print(f"索引已保存到: {index_path}")

# 加载索引
loaded_index = faiss.read_index(index_path)
print(f"加载的索引中的向量数: {loaded_index.ntotal}")

# 验证索引一致性
query = test_vectors[:1]
dist1, idx1 = index.search(query, 5)
dist2, idx2 = loaded_index.search(query, 5)

print(f"原始索引结果: {idx1[0][:3]}")
print(f"加载索引结果: {idx2[0][:3]}")
print(f"结果一致: {np.array_equal(idx1, idx2)}")

In [ ]:
# 量化索引演示
print("\n量化索引演示:")
print("="*30)

# 创建PQ (Product Quantization) 索引
dimension = 64
m = 8  # PQ的子向量数
nbits = 8  # 每个子向量的比特数

test_vectors, _ = create_sample_vectors(5000, dimension)

# 原始Flat索引
index_flat = faiss.IndexFlatL2(dimension)
index_flat.add(test_vectors)

# PQ索引
index_pq = faiss.IndexPQ(dimension, m, nbits)
index_pq.train(test_vectors)
index_pq.add(test_vectors)

print(f"Flat索引大小: {index_flat.ntotal * dimension * 4 / 1024:.1f} KB")
print(f"PQ索引大小估计: {index_pq.ntotal * m * nbits / 8 / 1024:.1f} KB")
print(f"压缩比: {(index_flat.ntotal * dimension * 4) / (index_pq.ntotal * m * nbits / 8):.1f}x")

# 比较搜索质量
query = test_vectors[:10]
k = 10

# Flat搜索
dist_flat, idx_flat = index_flat.search(query, k)

# PQ搜索
dist_pq, idx_pq = index_pq.search(query, k)

# 计算召回率
recall = calculate_recall(idx_flat, idx_pq, k)
print(f"PQ索引召回率@{k}: {recall:.3f}")

## 8. 最佳实践和优化建议

总结FAISS使用的最佳实践。

In [ ]:
print("FAISS最佳实践总结:")
print("="*50)

recommendations = {
    "数据规模": {
        "< 10K向量": "使用IndexFlatL2，简单快速",
        "10K - 100K向量": "使用IndexIVFFlat，平衡速度和精度",
        "100K - 1M向量": "使用IndexHNSWFlat或IndexIVFPQ",
        "> 1M向量": "使用IndexIVFPQ或分布式方案"
    },
    "精度要求": {
        "精确搜索": "IndexFlatL2",
        "高精度近似": "IndexHNSWFlat",
        "平衡精度速度": "IndexIVFFlat",
        "优先速度": "IndexIVFPQ"
    },
    "内存限制": {
        "内存充足": "IndexFlatL2或IndexHNSWFlat",
        "内存有限": "IndexIVFPQ或IndexPQ",
        "极限压缩": "IndexPQ with small m"
    }
}

for category, items in recommendations.items():
    print(f"\n{category}:")
    for scenario, recommendation in items.items():
        print(f"  {scenario}: {recommendation}")

print("\n\n优化技巧:")
tips = [
    "1. 向量归一化：对于余弦相似度，使用归一化向量",
    "2. 批量操作：批量添加向量和批量查询以提高效率",
    "3. GPU加速：对于大规模数据，考虑使用faiss-gpu",
    "4. 索引调优：根据数据分布调整nlist、nprobe等参数",
    "5. 内存映射：对于只读索引，使用内存映射减少内存使用"
]

for tip in tips:
    print(f"  {tip}")

In [ ]:
# 清理临时文件
import os
if os.path.exists('../data/faiss_index.bin'):
    os.remove('../data/faiss_index.bin')
    print("已清理临时索引文件")

print("\n演示完成！")
print("FAISS是一个强大的向量搜索库，适合各种规模的应用。")
print("选择合适的索引类型和参数对于获得最佳性能至关重要。")